# Analyse WandB Pretraining Data
resources used for this notebook:
* https://docs.wandb.ai/guides/track/public-api-guide/
* https://docs.wandb.ai/ref/python/public-api/runs/

In [1]:
import wandb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

In [2]:
api = wandb.Api()

# Finetuning Results
adapted from: https://docs.wandb.ai/guides/track/public-api-guide/ (22.07.2025)

## Metrics

In [3]:
entity, project = "josefine-busch-htw-berlin", "ma_finetuning"
runs = api.runs(f"{entity}/{project}")

In [4]:
names, tags, test_accuracies, test_losses, test_f1, test_precision, test_recall, test_mcc = [], [], [], [], [], [], [], []
for run in runs:
	names.append(run.name)
	tags.append(run.tags)
	test_accuracies.append(run.summary.get("test/accuracy", np.nan))
	test_losses.append(run.summary.get("test/loss", np.nan))
	test_f1.append(run.summary.get("test/f1", np.nan))
	test_precision.append(run.summary.get("test/precision", np.nan))
	test_recall.append(run.summary.get("test/recall", np.nan))
	test_mcc.append(run.summary.get("test/matthews_correlation", np.nan))

runs_df = pd.DataFrame(
	{
	"run_name": names,
	"test_accuracy": test_accuracies, "test_loss": test_losses, "test_f1": test_f1,
	"test_precision": test_precision, "test_recall": test_recall, "test_mcc": test_mcc}
)
runs_df

,run_name,test_accuracy,test_loss,test_f1,test_precision,test_recall,test_mcc
0,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_al...,0.781757,0.472779,0.781689,0.782212,0.781810,0.564022
1,HF_HS_5M_NRC_150K_42_v6_3e-5_prom_prom_core_al...,0.761486,0.491612,0.761060,0.763607,0.761606,0.525209
2,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_no...,0.790089,0.469136,0.789909,0.790954,0.790031,0.580984
3,HF_HS_5M_NRC_150K_42_v6_3e-5_prom_prom_core_no...,0.788958,0.482180,0.788673,0.790696,0.789038,0.579731
4,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_ta...,0.676998,0.587859,0.672606,0.688836,0.677988,0.366663
...,...,...,...,...,...,...,...
310,HF_HF_5M_RC_150K_3_v11_3e-5_prom_prom_300_all_...,0.883277,0.373002,0.882915,0.887143,0.882942,0.770074
311,HF_HF_5M_RC_150K_42_v11_3e-5_prom_prom_300_tat...,0.621533,0.678970,0.619100,0.628663,0.624230,0.252854
312,HF_HF_5M_RC_150K_199_v11_3e-5_prom_prom_300_ta...,0.639478,0.674505,0.638230,0.644538,0.641600,0.286123
313,HF_HF_5M_RC_150K_3_v11_3e-5_prom_prom_300_nota...,0.918787,0.270507,0.918784,0.919091,0.918949,0.838040


In [5]:
tags_df = pd.DataFrame(tags, columns=['job_id', 'tokenizer', 'task', 'subtask', 'scenario'])
tagged_runs = pd.concat([runs_df, tags_df], axis=1)
tagged_runs.head()

,run_name,test_accuracy,test_loss,test_f1,test_precision,test_recall,test_mcc,job_id,tokenizer,task,subtask,scenario
0,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_al...,0.781757,0.472779,0.781689,0.782212,0.781810,0.564022,14290,hf_5M_tokenizer,prom_core,prom_core_all,scenario1
1,HF_HS_5M_NRC_150K_42_v6_3e-5_prom_prom_core_al...,0.761486,0.491612,0.761060,0.763607,0.761606,0.525209,14293,hs_5M_tokenizer,prom_core,prom_core_all,scenario1
2,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_no...,0.790089,0.469136,0.789909,0.790954,0.790031,0.580984,14290,hf_5M_tokenizer,prom_core,prom_core_notata,scenario1
3,HF_HS_5M_NRC_150K_42_v6_3e-5_prom_prom_core_no...,0.788958,0.482180,0.788673,0.790696,0.789038,0.579731,14293,hs_5M_tokenizer,prom_core,prom_core_notata,scenario1
4,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_ta...,0.676998,0.587859,0.672606,0.688836,0.677988,0.366663,14290,hf_5M_tokenizer,prom_core,prom_core_tata,scenario1


In [7]:
mean_subtask_metrics = tagged_runs.groupby(['task', 'subtask', 'tokenizer']).agg({
	"test_accuracy": "mean", 
	"test_loss": "mean", 
	"test_f1": "mean", 
	"test_precision": "mean", 
	"test_recall": "mean", 
	"test_mcc": "mean"
})
mean_subtask_metrics

test_accuracy  test_loss  \
task      subtask          tokenizer                                      
p_covid   p_covid_sub      hf_2_5M_tokenizer        0.725907   0.742033   
                           hf_5M_rc_tokenizer       0.726670   0.745418   
                           hf_5M_tokenizer          0.726403   0.742635   
                           hs_2_5M_tokenizer        0.716138   0.758272   
                           hs_5M_tokenizer          0.714854   0.758530   
prom_300  prom_300_all     hf_2_5M_tokenizer        0.871378   0.352145   
                           hf_5M_rc_tokenizer       0.878510   0.338742   
                           hf_5M_tokenizer          0.873874   0.349040   
                           hs_2_5M_tokenizer        0.869163   0.338676   
                           hs_5M_tokenizer          0.868919   0.339153   
          prom_300_notata  hf_2_5M_tokenizer        0.915688   0.262344   
                           hf_5M_rc_tokenizer       0.915123   0.259944   
                           hf_5M_tokenizer          0.912296   0.267181   
                           hs_2_5M_tokenizer        0.909239   0.267065   
                           hs_5M_tokenizer          0.910600   0.266214   
          prom_300_tata    hf_2_5M_tokenizer        0.627877   0.659496   
                           hf_5M_rc_tokenizer       0.632409   0.664721   
                           hf_5M_tokenizer          0.628784   0.656574   
                           hs_2_5M_tokenizer        0.617002   0.656175   
                           hs_5M_tokenizer          0.617183   0.654632   
prom_core prom_core_all    hf_2_5M_tokenizer        0.775473   0.484761   
                           hf_5M_rc_tokenizer       0.777290   0.485933   
                           hf_5M_tokenizer          0.777346   0.481766   
                           hs_2_5M_tokenizer        0.764337   0.491055   
                           hs_5M_tokenizer          0.765803   0.498189   
          prom_core_notata hf_2_5M_tokenizer        0.787790   0.466253   
                           hf_5M_rc_tokenizer       0.786132   0.464216   
                           hf_5M_tokenizer          0.786048   0.467033   
                           hs_2_5M_tokenizer        0.790277   0.474335   
                           hs_5M_tokenizer          0.784813   0.471370   
          prom_core_tata   hf_2_5M_tokenizer        0.704405   0.585505   
                           hf_5M_rc_tokenizer       0.710350   0.583134   
                           hf_5M_tokenizer          0.701468   0.575317   
                           hs_2_5M_tokenizer        0.704527   0.610615   
                           hs_5M_tokenizer          0.702374   0.614878   

                                                test_f1  test_precision  \
task      subtask          tokenizer                                      
p_covid   p_covid_sub      hf_2_5M_tokenizer   0.728841        0.737226   
                           hf_5M_rc_tokenizer  0.730243        0.739790   
                           hf_5M_tokenizer     0.729169        0.737896   
                           hs_2_5M_tokenizer   0.719503        0.730047   
                           hs_5M_tokenizer     0.718047        0.728335   
prom_300  prom_300_all     hf_2_5M_tokenizer   0.871338        0.871839   
                           hf_5M_rc_tokenizer  0.878417        0.879330   
                           hf_5M_tokenizer     0.873853        0.874168   
                           hs_2_5M_tokenizer   0.869140        0.869476   
                           hs_5M_tokenizer     0.868884        0.869356   
          prom_300_notata  hf_2_5M_tokenizer   0.915669        0.916373   
                           hf_5M_rc_tokenizer  0.915094        0.916113   
                           hf_5M_tokenizer     0.912253        0.913449   
                           hs_2_5M_tokenizer   0.909214        0.910094   
                           hs_5M_tokenizer     0.910579        0.911293

In [23]:
tasks = ['prom_300', 'prom_core']
subtasks = ['_all', '_notata', '_tata']
columns = ['test_accuracy', 'test_loss', 'test_f1', 'test_precision', 'test_recall', 'test_mcc']

In [33]:
for task in tasks:
	print(f"Task: {task}")
	for subtask in subtasks:
		print(f" Subtask: {subtask}")
		for col in columns:
			delta = mean_subtask_metrics.loc[task].loc[f'{task}{subtask}'][col].max() - mean_subtask_metrics.loc[task].loc[f'{task}{subtask}'][col].min()
			print(f"Delta {col}: {delta:.4f}")

print('Covid')
for col in columns:
	delta = mean_subtask_metrics.loc['p_covid'].loc['p_covid_sub'][col].max()-mean_subtask_metrics.loc['p_covid'].loc['p_covid_sub'][col].min()
	print(f"Delta {col}: {delta:.4f}")

Task: prom_300
 Subtask: _all
Delta test_accuracy: 0.0096
Delta test_loss: 0.0135
Delta test_f1: 0.0095
Delta test_precision: 0.0100
Delta test_recall: 0.0094
Delta test_mcc: 0.0194
 Subtask: _notata
Delta test_accuracy: 0.0064
Delta test_loss: 0.0072
Delta test_f1: 0.0065
Delta test_precision: 0.0063
Delta test_recall: 0.0064
Delta test_mcc: 0.0127
 Subtask: _tata
Delta test_accuracy: 0.0154
Delta test_loss: 0.0101
Delta test_f1: 0.0310
Delta test_precision: 0.0080
Delta test_recall: 0.0132
Delta test_mcc: 0.0169
Task: prom_core
 Subtask: _all
Delta test_accuracy: 0.0130
Delta test_loss: 0.0164
Delta test_f1: 0.0136
Delta test_precision: 0.0116
Delta test_recall: 0.0129
Delta test_mcc: 0.0245
 Subtask: _notata
Delta test_accuracy: 0.0055
Delta test_loss: 0.0101
Delta test_f1: 0.0054
Delta test_precision: 0.0057
Delta test_recall: 0.0055
Delta test_mcc: 0.0112
 Subtask: _tata
Delta test_accuracy: 0.0089
Delta test_loss: 0.0396
Delta test_f1: 0.0071
Delta test_precision: 0.0148
Delta te

In [31]:
df = px.data.iris()
df

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3


In [29]:
fig = px.box(df, y="test_accuracy")

fig.show()

In [28]:
df = mean_subtask_metrics.loc['p_covid'].loc['p_covid_sub']
fig = px.violin(df, y="test_accuracy")
fig.show()

In [15]:
mean_subtask_metrics.loc['prom_300'].loc['prom_300_all']['test_accuracy'].min()

np.float64(0.8689189189189189)

In [ ]:
mean_subtask_metrics.to_csv("mean_finetuning_subtask_metrics.csv")

In [14]:
mean_task_metrics = tagged_runs.groupby(['scenario', 'task', 'tokenizer']).agg({
	"test_accuracy": "mean", 
	"test_loss": "mean", 
	"test_f1": "mean", 
	"test_precision": "mean", 
	"test_recall": "mean", 
	"test_mcc": "mean"
})
mean_task_metrics

test_accuracy  test_loss   test_f1  \
scenario  task      tokenizer                                                
scenario1 p_covid   hf_5M_tokenizer          0.726403   0.742635  0.729169   
                    hs_5M_tokenizer          0.714854   0.758530  0.718047   
          prom_300  hf_5M_tokenizer          0.804985   0.424265  0.802873   
                    hs_5M_tokenizer          0.798901   0.420000  0.793259   
          prom_core hf_5M_tokenizer          0.754954   0.508039  0.754328   
                    hs_5M_tokenizer          0.750997   0.528146  0.750395   
scenario2 p_covid   hf_2_5M_tokenizer        0.725907   0.742033  0.728841   
                    hf_5M_rc_tokenizer       0.726670   0.745418  0.730243   
                    hs_2_5M_tokenizer        0.716138   0.758272  0.719503   
          prom_300  hf_2_5M_tokenizer        0.804981   0.424662  0.803344   
                    hf_5M_rc_tokenizer       0.808680   0.421136  0.808260   
                    hs_2_5M_tokenizer        0.798468   0.420639  0.793143   
          prom_core hf_2_5M_tokenizer        0.755889   0.512173  0.755071   
                    hf_5M_rc_tokenizer       0.757924   0.511094  0.756661   
                    hs_2_5M_tokenizer        0.753047   0.525335  0.752581   

                                        test_precision  test_recall  test_mcc  
scenario  task      tokenizer                                                  
scenario1 p_covid   hf_5M_tokenizer           0.737896     0.725061  0.691395  
                    hs_5M_tokenizer           0.728335     0.713597  0.678488  
          prom_300  hf_5M_tokenizer           0.810448     0.806285  0.616613  
                    hs_5M_tokenizer           0.807682     0.800278  0.607208  
          prom_core hf_5M_tokenizer           0.757239     0.755116  0.512338  
                    hs_5M_tokenizer           0.752878     0.751019  0.503885  
scenario2 p_covid   hf_2_5M_tokenizer         0.737226     0.724557  0.690772  
                    hf_5M_rc_tokenizer        0.739790     0.725364  0.691687  
                    hs_2_5M_tokenizer         0.730047     0.714839  0.679951  
          prom_300  hf_2_5M_tokenizer         0.809089     0.806014  0.615019  
                    hf_5M_rc_tokenizer        0.810398     0.809134  0.619527  
                    hs_2_5M_tokenizer         0.806677     0.799703  0.605719  
          prom_core hf_2_5M_tokenizer         0.758545     0.756021  0.514525  
                    hf_5M_rc_tokenizer        0.762300     0.758046  0.520291  
                    hs_2_5M_tokenizer         0.754858     0.753061  0.507912

In [15]:
mean_task_metrics.to_csv("mean_finetuning_task_metrics.csv")

In [9]:
mean_metrics = pd.read_csv("mean_metrics.csv", index_col=[0, 1, 2, 3])

## Runtime

In [3]:
entity, project = "josefine-busch-htw-berlin", "ma_finetuning"
runs = api.runs(f"{entity}/{project}")

In [4]:
names, tags, eval_runtime, train_runtime = [], [], [], []
for run in runs:
	names.append(run.name)
	tags.append(run.tags)
	eval_runtime.append(run.summary.get("eval/runtime", np.nan))
	train_runtime.append(run.summary.get("train_runtime", np.nan))

runs_df = pd.DataFrame(
	{
	"run_name": names,
	"eval_runtime": eval_runtime,
	"train_runtime": train_runtime
}
)

In [5]:
tags_df = pd.DataFrame(tags, columns=['job_id', 'tokenizer', 'task', 'subtask', 'scenario'])
tagged_runs = pd.concat([runs_df, tags_df], axis=1)
tagged_runs.head()

,run_name,eval_runtime,train_runtime,job_id,tokenizer,task,subtask,scenario
0,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_al...,4.2485,829.0819,14290,hf_5M_tokenizer,prom_core,prom_core_all,scenario1
1,HF_HS_5M_NRC_150K_42_v6_3e-5_prom_prom_core_al...,4.1702,828.8850,14293,hs_5M_tokenizer,prom_core,prom_core_all,scenario1
2,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_no...,3.9872,755.6423,14290,hf_5M_tokenizer,prom_core,prom_core_notata,scenario1
3,HF_HS_5M_NRC_150K_42_v6_3e-5_prom_prom_core_no...,3.7640,746.8029,14293,hs_5M_tokenizer,prom_core,prom_core_notata,scenario1
4,HF_HF_5M_NRC_150K_42_v9_3e-5_prom_prom_core_ta...,0.4620,254.2423,14290,hf_5M_tokenizer,prom_core,prom_core_tata,scenario1


In [6]:
mean_runtimes = tagged_runs.groupby(['scenario', 'task', 'subtask', 'tokenizer']).agg({
	"eval_runtime": "mean",
	"train_runtime": "mean"
})

mean_runtimes

eval_runtime  \
scenario  task      subtask          tokenizer                          
scenario1 prom_300  prom_300_all     hf_5M_tokenizer         4.371867   
                                     hs_5M_tokenizer         4.216033   
                    prom_300_notata  hf_5M_tokenizer         3.841800   
                                     hs_5M_tokenizer         3.808033   
                    prom_300_tata    hf_5M_tokenizer         0.459900   
                                     hs_5M_tokenizer         0.457133   
          prom_core prom_core_all    hf_5M_tokenizer         4.191900   
                                     hs_5M_tokenizer         4.178767   
                    prom_core_notata hf_5M_tokenizer         3.852000   
                                     hs_5M_tokenizer         3.805200   
                    prom_core_tata   hf_5M_tokenizer         0.453067   
                                     hs_5M_tokenizer         0.451233   
scenario2 prom_300  prom_300_all     hf_2_5M_tokenizer       4.312250   
                                     hf_5M_rc_tokenizer      4.309700   
                                     hs_2_5M_tokenizer       4.226033   
                    prom_300_notata  hf_2_5M_tokenizer       3.835950   
                                     hf_5M_rc_tokenizer      3.800267   
                                     hs_2_5M_tokenizer       3.803800   
                    prom_300_tata    hf_2_5M_tokenizer       0.451300   
                                     hf_5M_rc_tokenizer      0.450800   
                                     hs_2_5M_tokenizer       0.456067   
          prom_core prom_core_all    hf_2_5M_tokenizer       4.190000   
                                     hf_5M_rc_tokenizer      4.138467   
                                     hs_2_5M_tokenizer       4.157067   
                    prom_core_notata hf_2_5M_tokenizer       3.691000   
                                     hf_5M_rc_tokenizer      3.723667   
                                     hs_2_5M_tokenizer       3.741533   
                    prom_core_tata   hf_2_5M_tokenizer       0.446750   
                                     hf_5M_rc_tokenizer      0.442833   
                                     hs_2_5M_tokenizer       0.450200   

                                                         train_runtime  
scenario  task      subtask          tokenizer                          
scenario1 prom_300  prom_300_all     hf_5M_tokenizer        852.207767  
                                     hs_5M_tokenizer        840.394500  
                    prom_300_notata  hf_5M_tokenizer        751.551800  
                                     hs_5M_tokenizer        758.543167  
                    prom_300_tata    hf_5M_tokenizer        252.795867  
                                     hs_5M_tokenizer        251.419033  
          prom_core prom_core_all    hf_5M_tokenizer        830.518833  
                                     hs_5M_tokenizer        825.373133  
                    prom_core_notata hf_5M_tokenizer        751.597700  
                                     hs_5M_tokenizer        745.836033  
                    prom_core_tata   hf_5M_tokenizer        251.439633  
                                     hs_5M_tokenizer        249.886267  
scenario2 prom_300  prom_300_all     hf_2_5M_tokenizer      837.710950  
                                     hf_5M_rc_tokenizer     834.246133  
                                     hs_2_5M_tokenizer      837.304933  
                    prom_300_notata  hf_2_5M_tokenizer      745.566250  
                                     hf_5M_rc_tokenizer     738.036867  
                                     hs_2_5M_tokenizer      755.752033  
                    prom_300_tata    hf_2_5M_tokenizer      248.578350  
                                     hf_5M_rc_tokenizer     247.341633  
                                     hs_2_5M_tokenizer      252.326833  
          prom_core prom_core_all    hf_2